<h1 style="color:MediumSeaGreen;">HW5 Problem 3</h1>

<h2 style="color:MediumSeaGreen;">Comparing different quantum architectures at the CHSH game</h2>

In this problem, you will investigate how well different quantum architecures are capable of winning the CHSH game.

(a) As we have seen in lecture, Qiskit allows you create a circuit consisting of the most commonly used quantum gates (e.g. X,Y,Z,CNOT etc.). Beyond this, Qiskit also allows you to arbitrarily specify the entries of a unitary matrix that you wish to insert in your circuit, for example using the Python package Numpy, and it will then figure out behind the scenes how best to implement the specified unitary using its native gates). 

Recall that in the CHSH game's optimal strategy that we saw in class, Bob makes a measurement in the basis $\mathcal{B}_0 = \{\cos(\frac{\pi}{8}) |0\rangle +\sin(\frac{\pi}{8}) |1\rangle, -\sin(\frac{\pi}{8}) |0\rangle +\cos(\frac{\pi}{8}) |1\rangle\}$ if he receives question 0, and a measurement in the basis $\mathcal{B}_1 = \{\cos(\frac{-\pi}{8}) |0\rangle +\sin(\frac{-\pi}{8}) |1\rangle, -\sin(\frac{-\pi}{8}) |0\rangle +\cos(\frac{-\pi}{8}) |1\rangle\}$ if he receives question 1. Moreover, recall from HW2 (Problem 1) that measuring in a basis is equivalent to applying an appropriate unitary transformation followed by a measurement in the standard basis. Let $B_0$ and $B_1$ be these unitary transformations for the measurements in the bases $\mathcal{B}_0$ and $\mathcal{B}_1$ respectively. In the following cell, define Numpy arrays corresponding to the matrix representations of $B_0$ and $B_1$. You may find the comments in the code helpful to guide you in the right direction if you are not familiar with Numpy.

In [ ]:
import numpy as np
import math

theta_0 = math.pi/8
theta_1 = -math.pi/8

A_0 = np.array([[1, 0], [0, 1]])
A_1 = np.array([[1, 1], [1, -1]]) / math.sqrt(2)

#In numpy, you can take cosine and sine of a number x as "np.cos(x)", "np.sin(x)".
#The syntax for defining your matrix matrices should be, e.g.
#B_0 = np.array([[a11, a12], [a21, a22]])
#where a11, a12, a21, a22 are the entries.
B_0 = np.array([[np.cos(theta_0), np.sin(theta_0)], [-np.sin(theta_0), np.cos(theta_0)]])
B_1 = np.array([[np.cos(theta_1), np.sin(theta_1)], [-np.sin(theta_1), np.cos(theta_1)]])

(b) In the next cell, you will execute many iterations of the CHSH game (using IBM's AerSimulator, which we also used at the start of the First_quantum_program notebook), and at the end compute and print the fraction of times the game was won. For each iteration, you will:

(i) Create an EPR pair;

(ii) Sample Alice and Bob's question uniformly at random;

(iii) Perform Alice and Bob's actions (implemented as an appropriate unitary followed by a measurement in the standard basis).

The cell contains comments and bits and pieces of the code to guide you in the right direction.

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import random

N = 1000 #number of games played

games_won = 0
sim = AerSimulator()

for i in range(N):              #We will play the game N times
    
    circ = QuantumCircuit(2, 2) #Define a quantum circuit 'circ' on two qubits that creates an EPR pair
    circ.h(0)
    circ.cx(0, 1)    
    
    x = random.randint(0,1)      #sampling a question for Alice
    y = random.randint(0,1)      #sampling a question for Bob
    
    if x == 0:                   
        circ.unitary(A_0, [0], label = 'A_0') #add to the previous circuit the appropriate unitary that Alice applies on question x=0
    else:                        
        circ.unitary(A_1, [0], label = 'A_1') #add to the previous circuit the appropriate unitary that Alice applies on question x=1
        
    if y == 0:       #add Bob's unitary gate B_0, which you have defined in the previous cell (this is already added for you)
        circ.unitary(B_0, [1], label = 'B_0')
    else:            #add Bob's unitary gate B_1, which you have defined in the previous cell (this is already added for you)
        circ.unitary(B_1, [1], label = 'B_1')
    
    
    #add a measurement of both qubits to your circuit
    circ.measure([0, 1], [0, 1])
    
    #compile circuit in terms of IBM's native gates.
    transp_circ = transpile(circ, basis_gates=['rx', 'ry', 'cx'])
    
    #execute ONE SHOT of your circuit transp_circ using IBM's "AerSimulator". 
    job = sim.run(transp_circ, shots=1)

    #get the result of the simulation
    sim_result = job.result()
    
    #get the outcomes
    counts = sim_result.get_counts(transp_circ)
    a = int(list(dict.keys(counts))[0][0])      # Alice's answer
    b = int(list(dict.keys(counts))[0][1])      # Bob's answer

    if (a^b) == (x & y): #check the winning condition, and keep track of the information of whether they won or lost
        games_won += 1
    
#print the ratio of games that were won to the total games played N
print("Ratio of games won to total games played: ", games_won/N)

If you implemented your code correctly, you should have observed that the CHSH game is won roughly 0.85 fraction of the times. In the rest of the question, you will test your code on actual quantum devices to see how they perform!

(c) In the next cell, modify your code so that the questions $x,y$ are not sampled *within* each iteration. Instead, for each of the four possible pairs of questions $(x,y)$:

(i) Build the quantum circuit corresponding to the questions $(x,y)$;

(ii) Run 250 shots of that quantum circuit (again using the same simulator);

(iii) Compute and print the fraction of times the game was won.

Verify that, for each of the possible question pairs, the fraction of times that the game is won is approximately $0.85$.

This modification is necessary so that the cost of running your program on actual quantum devices does not become exorbitant! Please implement this modification very carefully! :) And only proceed further if this part works correctly (otherwise you will burn your precious credits).

In [ ]:
#your code here
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

#Define the simulator
sim = AerSimulator()

#Define the number of shots per question
shots = 250

#Initialize the dictionary to store the results
results = {}

#Iterate over all possible questions for Alice and Bob
for x in [0, 1]:
    for y in [0, 1]:
        #initialize the count of games won for this pair of questions
        games_won = 0

        #Play the game 250 times
        for i in range(shots):
            #Define the quantum circuit
            circ = QuantumCircuit(2, 2)
            circ.h(0)
            circ.cx(0, 1)
            if x == 0:
                circ.unitary(A_0, [0], label = 'A_0')
            else:
                circ.unitary(A_1, [0], label = 'A_1')
            if y == 0:
                circ.unitary(B_0, [1], label = 'B_0')
            else:
                circ.unitary(B_1, [1], label = 'B_1')
            circ.measure([0, 1], [0, 1])

            #Compile the circuit
            transp_circ = transpile(circ, basis_gates=['rx', 'ry', 'cx'])
            job = sim.run(transp_circ, shots=1)
            sim_result = job.result()
            counts = sim_result.get_counts(transp_circ)
            a = int(list(dict.keys(counts))[0][0])
            b = int(list(dict.keys(counts))[0][1])

            #Check if the winning condition is satisfied
            if (a^b) == (x & y):
                games_won += 1

        #Store the results in the dictionary
        results[(x, y)] = games_won/shots

#Print the results
for (x, y) in results:
    print("Alice's question: ", x, ", Bob's question: ", y, ", Ratio of games won: ", results[(x, y)])

(d) In this part you will modify your code from part (c) so that the call to the 'AerSimulator' is replaced by a call to an actual quantum device! You will run the same circuit on multiple quantum devices. Run the following cell first to obtain a list of all supported devices and their current status (i.e. whether they are available for a quantum circuit to be submitted).

In [ ]:
from qbraid import get_devices

get_devices()

Choose one of the available devices and create a "device" object corresponding to that device id by running the following cells:

In [ ]:
from qbraid.providers import QbraidProvider

token = "YOUR_TOKEN" #replace "
provider = QbraidProvider(qiskit_ibm_token=token)
qbraid_id = 'ibm_q_brisbane'
device = provider.get_device(qbraid_id)
job = device.run(circ, shots=250)

In [ ]:
qbraid_id = #insert your chosen device id
device = provider.get_device(qbraid_id)

Now, modify your code from part (c) so that, for each of the four pairs of questions, the corresponding quantum circuit is executed on a quantum device of your choice (with 250 shots). Plot your results. Do the same for a second quantum device of your choice. (Some quantum devices are more expensive than others: you may wish to avoid running on ionQ for this part!)

In [ ]:
#your code

What is the average CHSH winning probability (across the four possible question pairs) that you observed for each device?

(Again, you may find it helpful to look back at the First_quantum_program notebook for the syntax of how to extract measurement counts.)

(e) (optional) In the current code, we compiled the quantum circuit in terms of a set of native quantum gates that are supported by IBM's quantum computers. Does the performance on other quantum devices improve if you instead compile your circuit with respect to a different set of gates that is native to those quantum devices?